In [70]:
import os
import torch
import torch.nn.functional as F
import torch_directml
import wandb
import numpy as np
from tqdm import tqdm
import h5py
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import dense_to_sparse
from torch_geometric.utils import dense_to_sparse, add_self_loops


In [71]:
# ✅ Initialize Weights & Biases (W&B)
wandb.init(project="higgs_gnn", name="gnn_training_final", config={"epochs": 20, "batch_size": 64})

In [72]:
# ✅ Set device to AMD GPU (DirectML)
device = torch_directml.device()
print(f"✅ Using device: {device}")

✅ Using device: privateuseone:0


In [73]:
# Load HDF5 dataset
import h5py
hdf5_file = r"C:\Users\vudut\OneDrive\Desktop\Python\MINI Project\Data Sets\jet-images_Mass60-100_pT250-300_R1.25_Pix25.hdf5"

with h5py.File(hdf5_file, "r") as f:
    jet_pt = np.array(f["jet_pt"])
    jet_eta = np.array(f["jet_eta"])
    jet_phi = np.array(f["jet_phi"])  # Added
    jet_mass = np.array(f["jet_mass"])  # Added
    signal = np.array(f["signal"])  # Labels: 1 = Signal, 0 = Background

In [74]:
# Normalize features
features = np.stack([jet_pt, jet_eta, jet_phi, jet_mass], axis=1)  # Expanded feature set
features = (features - features.mean(axis=0)) / features.std(axis=0)  # Standardization

In [75]:
# Create Graph Data (Fully Connected Graph)
graphs = []
for i in tqdm(range(len(features)), desc="Creating Graphs"):
    # Node features
    x = torch.tensor(features[i], dtype=torch.float).unsqueeze(0)  
    # Label
    y = torch.tensor([int(signal[i])], dtype=torch.long)
    
    # Create edge index for a single node (self-loop)
    # Using torch.zeros instead of just [0,0] to ensure proper device placement
    edge_index = torch.zeros((2, 1), dtype=torch.long)
    
    # Create graph object
    graph = Data(x=x, edge_index=edge_index, y=y)
    graphs.append(graph)

Creating Graphs: 100%|██████████| 872666/872666 [01:15<00:00, 11514.41it/s]


In [76]:
# Create DataLoader with additional parameters for stability
batch_size = 32  # Reduced batch size for better stability
train_loader = DataLoader(
    graphs[:int(0.8 * len(graphs))],
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  # Disable multi-processing to avoid potential issues
    pin_memory=True,  # Enable faster data transfer to GPU
    drop_last=True  # Drop the last incomplete batch to ensure consistent batch sizes
)

val_loader = DataLoader(
    graphs[int(0.8 * len(graphs)):],
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    drop_last=False  # Keep all validation samples
)

c:\Users\vudut\anaconda3\envs\pytorchamd\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [77]:
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        # Initialize with add_self_loops=True since we want to include self-loops
        self.conv1 = GCNConv(4, 16, add_self_loops=True)
        self.conv2 = GCNConv(16, 32, add_self_loops=True)
        self.fc = torch.nn.Linear(32, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Ensure tensors are on the correct device
        x = x.to(edge_index.device)
        
        # Get batch information
        batch = data.batch if hasattr(data, 'batch') else torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        
        # Apply convolutions
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        
        # Global pooling and classification
        x = global_mean_pool(x, batch)
        return self.fc(x)

In [78]:
# Training Loop
model = GNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(20):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        try:
            # Move batch to device
            batch = batch.to(device)
            
            optimizer.zero_grad()
            out = model(batch)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            total += batch.y.size(0)
            
        except RuntimeError as e:
            print(f"Error in batch: {e}")
            continue
    
    if total > 0:
        train_acc = correct / total
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch}: Loss = {avg_loss:.4f}, Accuracy = {train_acc:.4f}")
        
        wandb.log({
            "epoch": epoch,
            "train_loss": avg_loss,
            "train_accuracy": train_acc
        })

Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is incorrect.
Error in batch: The parameter is

KeyboardInterrupt: 